In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
demand_data = pd.read_csv('data\dataset\demand.csv')
vehicle_data = pd.read_csv('data/dataset/vehicles.csv')
vehicle_fuels = pd.read_csv('data/dataset/vehicles_fuels.csv')
fuels_data = pd.read_csv('data/dataset/fuels.csv')
carbon_emissions_data = pd.read_csv('data/dataset/carbon_emissions.csv')
cost_profiles = pd.read_csv('data/dataset/cost_profiles.csv')

In [3]:
avail_vehicle = {}
buy = {}
use = {}
sell = {}

In [4]:
def feasible_vehicle(yr, s,d):
    temp_df = vehicle_data.loc[(vehicle_data['Year'] == yr) & (vehicle_data['Size'] == s)]
    if d == 'D2':
        temp_df = temp_df.loc[temp_df['Distance'] != 'D1']
    elif d == 'D3':
        temp_df = temp_df.loc[(temp_df['Distance'] == 'D3') | (temp_df['Distance'] == 'D4')]
    elif d == 'D4':
        temp_df = temp_df.loc[(temp_df['Distance'] == 'D4')]
    return temp_df

In [5]:
unique_year = np.unique(demand_data['Year'])
s = np.unique(demand_data['Size'])
d = np.unique(demand_data['Distance'])

In [6]:
#initial
year = 2023
init_demand = demand_data.loc[demand_data['Year'] == year]
for i in range(len(s)):
    for j in range(len(d)-1,-1,-1):
        demand = init_demand['Demand (km)'].loc[(init_demand['Size'] == s[i]) & (init_demand['Distance'] == d[j])].iloc[0]
        feas_veh = feasible_vehicle(year,s[i],d[j])

In [9]:
demand

14576